In [2]:
import scipy
import numpy as np
import matplotlib.pyplot as plt

In [9]:
def create_rectangular_mesh(num_x_elements, num_y_elements, x_bounds=(0, 1), y_bounds=(0, 1)):
    x_min, x_max = x_bounds
    y_min, y_max = y_bounds
    num_x = num_x_elements + 1
    num_y = num_y_elements + 1
   # Generate vertices
    x = np.linspace(x_min, x_max, num_x)
    y = np.linspace(y_min, y_max, num_y)
    vertices = np.array([(xi, yj) for yj in y for xi in x])

    # Generate triangles
    triangles = []
    for j in range(num_y_elements):
        for i in range(num_x_elements):
            # Define the indices of the vertices for the current quad
            v0 = i + j * num_x
            v1 = v0 + 1
            v2 = v0 + num_x
            v3 = v2 + 1

            if (i + j == 0) or (i + j == (num_x_elements + num_y_elements) - 2):
                triangles.append([v0, v1, v3])
                triangles.append([v0, v3, v2])
            else:
                triangles.append([v0, v1, v2])
                triangles.append([v1, v3, v2])

    return vertices, np.array(triangles)

In [3]:
def dJ( u, v, p1, p2, p3 ):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    dxdu = ( (1-v)*x2 + v*x3 - x1 )
    dxdv = ( u*x3 - u*x2 )
    dydu = ( (1-v)*y2 + v*y3 - y1 )
    dydv = ( u*y3 - u*y2 )
    return np.abs( dxdu*dydv - dxdv*dydu )

In [4]:
def tridblquad( integrand, p1, p2, p3 ):
    '''
    Perform double quadtrature integration on triangular domain.
    Input: function to integrate, points of triangle as tuples.
    Output: integral and estimated absolute error as a tuple.
    '''
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    # transformation to the unit square
    g = lambda u, v, c1, c2, c3: (1-u)*c1 + u*( (1-v)*c2 + v*c3 )
    # transformation for the integrand, 
    # including the Jacobian scaling factor
    def h( u, v ):
        x = g( u, v, x1, x2, x3 )
        y = g( u, v, y1, y2, y3 )
        I = integrand( x, y )
        I *= dJ( u, v, p1, p2, p3 )
        return I
    # perfrom the double integration using quadrature in the transformed space
    integral, error = scipy.integrate.dblquad( h, 0, 1, lambda x: 0, lambda x: 1, epsrel=1e-6, epsabs=0 )
    return integral, error

In [7]:
def integrate_over_mesh(integrand_func, num_x_elements=10, num_y_elements=10, x_bounds=(0, 1), y_bounds=(0, 1)):
    # Generate the rectangular mesh
    vertices, triangles = create_rectangular_mesh(num_x_elements, num_y_elements, x_bounds, y_bounds)

    # Perform integration over each triangle in the mesh
    total_integral = 0.0
    for triangle_indices in triangles:
        # Extract vertices of the current triangle
        p1 = vertices[triangle_indices[0]]
        p2 = vertices[triangle_indices[1]]
        p3 = vertices[triangle_indices[2]]

        # Calculate the integral over the current triangle using tridblquad
        integral, _ = tridblquad(integrand_func, p1, p2, p3)
        
        # Add the integral value to the total
        total_integral += integral

    return total_integral

In [18]:
func = lambda x, y: np.sqrt(3*x/4) + np.sin(y)**2
x_bounds = (0, 20)
y_bounds = (0, 20)
num_x_elements = 2
num_y_elements = 2

custom = integrate_over_mesh(func, num_x_elements, num_y_elements, x_bounds, y_bounds)
built, _ = scipy.integrate.dblquad(func, x_bounds[0], x_bounds[1], lambda x: y_bounds[0], lambda x: y_bounds[1])
print(custom, built)
print(np.abs(custom - built))

1229.0699930087699 1229.0699931862478
1.7747788660926744e-07


In [19]:
def lebesque_norm(func, num_x_elements=10, num_y_elements=10, x_bounds=(0, 1), y_bounds=(0, 1), triangulation=True):
    func_squared = lambda x, y: func(x, y)**2
    if triangulation:
        integral = integrate_over_mesh(func_squared, num_x_elements, num_y_elements, x_bounds, y_bounds)
    else:
        integral, _ = scipy.integrate.dblquad(func_squared, x_bounds[0], x_bounds[1], lambda x: y_bounds[0], lambda x: y_bounds[1])
    return np.sqrt(integral)

def sobolew_norm(func, x_der, y_der, num_x_elements=10, num_y_elements=10, x_bounds=(0, 1), y_bounds=(0, 1), triangulation=True):
    final_func = lambda x, y: func(x, y)**2 + x_der(x, y)**2 + y_der(x, y)**2
    if triangulation:
        integral = integrate_over_mesh(final_func, num_x_elements, num_y_elements, x_bounds, y_bounds)
    else:
        integral, _ = scipy.integrate.dblquad(final_func, x_bounds[0], x_bounds[1], lambda x: y_bounds[0], lambda x: y_bounds[1])
    return np.sqrt(integral)

In [21]:
func = lambda x, y: x*y
x_der = lambda x, y: y
y_der = lambda x, y: x
x_bounds = (0, 1)
y_bounds = (0, 1)
num_x_elements, num_y_elements = 10, 10

lebesque_norm_res = lebesque_norm(func, num_x_elements, num_y_elements, x_bounds, y_bounds)
print(f"Approximated Lebesque norm: {lebesque_norm_res}")
print(f"Exact Lebesque norm: {1/3}")
print(f"Error: {np.abs(lebesque_norm_res - np.sqrt(1/9))}\n")

sobolew_norm_res = sobolew_norm(func, x_der, y_der, num_x_elements, num_y_elements, x_bounds, y_bounds)
print(f"Approximated Sobolew norm: {sobolew_norm_res}")
print(f"Exact Sobolew norm: {np.sqrt(7)/3}")
print(f"Error: {np.abs(sobolew_norm_res - np.sqrt(7)/3)}")

Approximated Lebesque norm: 0.33333333333333337
Exact Lebesque norm: 0.3333333333333333
Error: 5.551115123125783e-17

Approximated Sobolew norm: 0.8819171036881972
Exact Sobolew norm: 0.8819171036881969
Error: 3.3306690738754696e-16
